In [1]:
!pip install keras-tuner
!pip install tensorflow-addons

In [2]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import librosa.display

from IPython.display import clear_output

from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
import tensorflow_addons as tfa
import keras_tuner as kt

In [4]:
def google_or_kaggle(string):
    string = string.lower()
    if string == 'google':
        from google.colab import drive
        drive.mount('/content/drive', force_remount=True)
        root_dir = '/content/drive/MyDrive/'
        base_dir = root_dir + 'Music Genre/mfcc/small/'
    elif string == 'kaggle':
         # Kaggle path
        base_dir = '/kaggle/input/fma-mel-new/mel/mel/'
    else:
        base_dir = ''

    return base_dir


In [5]:
base_dir = google_or_kaggle('kaggle') 

In [6]:
train_dir = base_dir + 'training/'
val_dir = base_dir + 'validation/'
test_dir = base_dir + 'test/'

In [7]:
from PIL import Image
from matplotlib import cm

mfcc_data = Image.open(train_dir + 'pop/000_000010_s0.png')

imarray = np.array(mfcc_data) 
mfcc_shape = imarray.shape
print( mfcc_shape )

ig, ax = plt.subplots()
mfcc_data= np.swapaxes(mfcc_data, 0 ,1)
cax = ax.imshow(mfcc_data, interpolation='nearest', cmap=cm.coolwarm, origin='lower', aspect='auto')
ax.set_title('MFCC')
#Showing mfcc_data
plt.show()

In [8]:
train_datagen = ImageDataGenerator(rescale= 1/255)
val_datagen = ImageDataGenerator(rescale= 1/255)
test_datagen = ImageDataGenerator(rescale= 1/255)

batch_size = 128

num_classes = 8
input_shape = (mfcc_shape[0], mfcc_shape[1], 1)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(input_shape[0], input_shape[1]),
        color_mode= "grayscale",
        batch_size=batch_size,
        class_mode='sparse')

validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(input_shape[0], input_shape[1]),
        color_mode= "grayscale",
        batch_size=batch_size,
        class_mode='sparse')

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(input_shape[0], input_shape[1]),
        color_mode= "grayscale",
        batch_size=batch_size,
        class_mode='sparse')

In [9]:
precision_function = tf.keras.metrics.Precision()
recall_function = tf.keras.metrics.Recall()
auc_function = tf.keras.metrics.AUC()
f1_function = tfa.metrics.F1Score(num_classes=num_classes, average='micro')

In [10]:
print( input_shape )

model = keras.Sequential(
    [
     keras.Input(shape=(input_shape)),
     layers.Resizing(128,128),


     layers.Conv2D(64, kernel_size=(5, 5), activation="relu"),
     layers.BatchNormalization(),
     layers.MaxPooling2D(pool_size=(2, 2)),

     layers.Conv2D(64, kernel_size=(5, 5), activation="relu"),
     layers.BatchNormalization(),
     layers.MaxPooling2D(pool_size=(2, 2)),

     layers.Conv2D(128, kernel_size=(5, 5), activation="relu"),
     layers.BatchNormalization(),
     layers.MaxPooling2D(pool_size=(2, 2)),

     layers.Conv2D(128, kernel_size=(5, 5), activation="relu"),
     layers.BatchNormalization(),
     layers.MaxPooling2D(pool_size=(2, 2)),


     layers.Flatten(), #flat the data
     layers.Dropout(0.2),
     layers.Dense(num_classes, activation="softmax")
    ]
)

model.summary()

model.compile( 
        loss="sparse_categorical_crossentropy", 
        metrics=["accuracy", f1_function],
        optimizer=keras.optimizers.Adam(learning_rate=0.001) )

In [11]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

In [12]:
# Train it on the data for some epochs
epochs = 50

history = model.fit(train_generator, epochs=epochs, validation_data=validation_generator, callbacks=[callback])

In [13]:
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

In [14]:
model.save('cnn_model_2021')

In [15]:
!zip -r file.zip '/kaggle/working/cnn_model_2021'

In [16]:
len( history.history['loss'] )

In [17]:
from matplotlib import pyplot as plt
#x_plot = list(range(1,epochs+1))

def plot_history(network_history):
    epochs = len( history.history['loss'] )
    x_plot = list(range(1,epochs+1))
    
    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.plot(x_plot, network_history.history['loss'])
    plt.plot(x_plot, network_history.history['val_loss'])
    plt.legend(['Training', 'Validation'])

    plt.figure()
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.plot(x_plot, network_history.history['accuracy'])
    plt.plot(x_plot, network_history.history['val_accuracy'])
    plt.legend(['Training', 'Validation'], loc='lower right')
    plt.show()

In [18]:
plot_history(history)

In [19]:
eval_result = model.evaluate(test_generator)
print("[test loss, test accuracy]:", eval_result)

In [20]:
test_pred = model.predict(test_generator)

In [21]:
y_pred = []
for pred in test_pred:
  y_pred.append( np.argmax(pred) )

In [22]:
from sklearn.metrics import f1_score

y_true = test_generator.labels

f1_test = f1_score(y_true, y_pred, average='macro')
print('Average f1_score: {} \n' .format(f1_test) )

print('F1-SCORE FOR EACH CLASS')
print('-----------------------')
av_f1_score = f1_score(y_true, y_pred, average=None)
for i in range(len(av_f1_score)):
  print('{} : {} '.format( i, av_f1_score[i]))

In [23]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay( confusion_matrix=cm )

disp.plot()
frame1 = plt.gca()
frame1.axes.get_xaxis().set_visible(False)
plt.show()